# Test

# 一開始先準備好原始 dataset 這是跨 project 的 common 部分

In [1]:
%run ../../../forth.py

reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



In [2]:
import pandas as pd
import numpy as np
import math

In [4]:
dataset_filename = 'Result_Rosa_DT -0623 Tool.xlsx'
raw_data = pd.read_excel(dataset_filename, sheet_name=0)  # 沒有指定就是 worksheet #0 
raw_data

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,BugId,Title,Status,P1(What_Happen),P2(What_Device),tool_P3(What_Environment),P4(Function)
0,PIMS-10011,MTP/SFF_SUT prompt not support after run comma...,Analyze,not support,NaN,tool,NaN
1,PIMS-10023,MTP/SFF_The error message appear during runing...,Analyze,still appears,"dgfx, vga",NaN,NaN
2,PIMS-10027,MTP/SFF_BU_'Storage' page show 2 M.2 PCIe SSD ...,Wait,shows in,"pcie/pci, dimm, storage",bios,NaN
3,PIMS-10041,MTE_SUT hang up with black screen and blinking...,Closed,"halt, auto reboot",NaN,NaN,boot event
4,PIMS-10120,MTP/SFF_Two NVME SSD password option show in B...,Closed,"shows in, show in, two nvme ssd password option","pcie/pci, storage",bios,password
...,...,...,...,...,...,...,...
1014,PIMS-7232,MTE_WS_Bitlocker cannot encrypt the system dri...,Closed,cannot encrypt,storage,NaN,storage
1015,PIMS-7343,"MFF, OpenCL Runtime Version doesn't meet SPEC ...",Closed,not meet,NaN,application,igcc
1016,PIMS-8318,"MFF,19042,20H2FI,Edge of Chrome launch automat...",Closed,launch automatically,NaN,"os, application, btoa",NaN
1017,PIMS-9100,MTP/SFF_RKL_Noise appear during playback video...,Analyze,still appears,"audio, dgfx, vga, cpu",NaN,play media


In [5]:
# James 有整理好各種 key names 放在另一張 work sheet #1
raw_targets = pd.read_excel(dataset_filename,sheet_name=1)
raw_targets

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,P1,P1_Count,P2,P2_Count,P3,P3_Count,P4,P4_Count
0,shows in,231,NaN,539.0,NaN,297.0,NaN,374.0
1,NaN,137,dgfx,82.0,bios,288.0,s3/s4/s5,94.0
2,fail,124,storage,80.0,os,146.0,bios menu,90.0
3,error message,75,usb device,56.0,tool,142.0,password,60.0
4,halt,75,cpu,50.0,s3/s4/s5,61.0,flash fw,43.0
...,...,...,...,...,...,...,...,...
254,can't generate,1,NaN,NaN,NaN,NaN,NaN,NaN
255,time out,1,NaN,NaN,NaN,NaN,NaN,NaN
256,keep refreshing under 4k resolution,1,NaN,NaN,NaN,NaN,NaN,NaN
257,"shows ""invalid attribute name: nvmepwdfeature""",1,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# P2 devices 的總集合
devices = raw_targets[['P2','P2_Count']].set_index('P2').to_dict()['P2_Count']
del(devices[np.nan]) # 其中有 NaN 項如上列，要注意。
%f devices count nip --> 1 ( count ) # devices 含 device name 與 sampled count 將來小於 20 的必須捨棄
for k,v in devices.items():
    if v >= 20 : 
        %f dup . space 1+ k . space v . cr


devices count nip --> 40 (<class 'int'>)
devices 含 device name 與 sampled count 將來小於 20 的必須捨棄

1 dgfx 82.0
2 storage 80.0
3 usb device 56.0
4 cpu 50.0
5 audio 36.0
6 dimm 35.0
7 tpm 33.0
8 nic/network 30.0
9 pcie/pci 26.0
10 vga 23.0
11 type-c 20.0
12 dp 20.0


現有的 project factory 只有以下 11 種 parser model 可用的用箭頭標註
    1 storage 103.0 <-------------------------
    2 usb device 78.0 <-------------------------
    3 type-c 75.0 <-------------------------
    4 audio 57.0 <-------------------------
    5 mouse/keyboard 55.0
    6 vga 52.0 <-------------------------
    7 tbt 39.0
    8 lcd 35.0
    9 battery 26.0
    10 cpu 25.0 <-------------------------
    11 dimm 24.0 <-------------------------

In [7]:
# 拆分 P2 成個別 device 的 list 準備作為 target column 
classes = []
for dd in raw_data['P2(What_Device)']:
    row = []
    for i in range(len(devices)):
        if type(dd) == str and list(devices.keys())[i] in dd:  # dd 有的是 np.nan 屬 float 要排除
            row.append(1)
        else:
            row.append(0)
    classes.append(row)
%f classes count nip -->
%f raw_data count nip -->
%f classes :> [7] -->

classes count nip --> 1019 (<class 'int'>)
raw_data count nip --> 1019 (<class 'int'>)
classes :> [7] --> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)


In [8]:
targets = pd.DataFrame(np.array(classes), columns = devices)  # 這裡挺妙，columns 直接餵 devices 即可，不必弄成 list(devices.keys()).
targets

,dgfx,storage,usb device,cpu,audio,dimm,tpm,nic/network,pcie/pci,vga,...,ac adapter,communications port,serial option card,miracast device,lid,touch lcd,optional module,tbt,iphone,dtd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1017,1,0,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data = pd.merge(raw_data['Title'],targets, left_index=True, right_index=True)
data 

,Title,dgfx,storage,usb device,cpu,audio,dimm,tpm,nic/network,pcie/pci,...,ac adapter,communications port,serial option card,miracast device,lid,touch lcd,optional module,tbt,iphone,dtd
0,MTP/SFF_SUT prompt not support after run comma...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MTP/SFF_The error message appear during runing...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MTP/SFF_BU_'Storage' page show 2 M.2 PCIe SSD ...,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,MTE_SUT hang up with black screen and blinking...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MTP/SFF_Two NVME SSD password option show in B...,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,MTE_WS_Bitlocker cannot encrypt the system dri...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1015,"MFF, OpenCL Runtime Version doesn't meet SPEC ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1016,"MFF,19042,20H2FI,Edge of Chrome launch automat...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1017,MTP/SFF_RKL_Noise appear during playback video...,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# 對照檢查無誤
data.to_excel("1.xls")

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  


In [10]:
import datarobot as dr

In [11]:
dr.Client(token=peforth.execute('api_token').pop(), endpoint=peforth.execute('endpoint').pop())

# 對某 device 發動專屬 project 做 prediction 並檢查對錯

符合 >20 samples 的 devices 有這些 

     1 storage       103.0
     2 usb device     78.0
     3 type-c         75.0
     4 audio          57.0
     5 mouse/keyboard 55.0
     6 vga            52.0
     7 tbt            39.0
     8 lcd            35.0
     9 battery        26.0
    10 cpu            25.0
    11 dimm           24.0


In [85]:
# 針對每一種 device 開特定 project 去分辨
project_name = 'dimm' 

In [86]:
project_id = dr.Project.list(search_params={'project_name':project_name})[0].id
project = dr.Project.get(project_id=project_id)
%f project_id -->
%f project -->


project_id --> 60d1ed7f3d8429e3f2cdb6e8 (<class 'str'>)
project --> Project(QT TR Title Parsing-dimm) (<class 'datarobot.models.project.Project'>)


- 如上述：不行，recommended model 可能都沒有 holdout 可用了。直接用 project.get_models() 好了。

In [87]:
models = project.get_models()
%f models :> [0] --> # The first one is the recommended model but probably is_frozen
%f models :> [0].is_frozen -->
%f models :> [1].is_frozen -->
%f models :> [2].is_frozen -->


models :> [0] --> Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)') (<class 'datarobot.models.model.Model'>)
The first one is the recommended model but probably is_frozen

models :> [0].is_frozen --> True (<class 'bool'>)
models :> [1].is_frozen --> False (<class 'bool'>)
models :> [2].is_frozen --> False (<class 'bool'>)


In [88]:
# Calculate new training predictions on holdout partition of dataset
training_predictions_job = models[1].request_training_predictions(dr.enums.DATA_SUBSET.HOLDOUT)
training_predictions = training_predictions_job.get_result_when_complete()
training_predictions


In [89]:
# Fetch training predictions as data frame
df_predict_holdout = training_predictions.get_all_as_dataframe()
df_predict_holdout

,row_id,partition_id,prediction,class_1.0,class_0.0
0,0,Holdout,0.0,0.000112,0.999888
1,4,Holdout,0.0,0.000153,0.999847
2,22,Holdout,0.0,0.000295,0.999705
3,29,Holdout,0.0,0.002666,0.997334
4,35,Holdout,0.0,0.001251,0.998749
...,...,...,...,...,...
278,1379,Holdout,0.0,0.004084,0.995916
279,1386,Holdout,0.0,0.006913,0.993087
280,1391,Holdout,0.0,0.001526,0.998474
281,1403,Holdout,0.0,0.000295,0.999705


In [90]:
# 瞧一瞧所有的 prediction 結果。等下要拿來跟 dataset 比對。
%f df_predict_holdout :> ['prediction'] list --> cr
%f df_predict_holdout :> ['row_id'] list -->
    

df_predict_holdout :> ['prediction'] list --> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [91]:
# prediction 結果跟 dataset 比對。用 prediction 0 的 index 去算 dataset 的 sum 就是 FN
indexs = list(df_predict_holdout.query('prediction == 0')['row_id'])
%f indexs count nip -->
positives = [data[project_name].iloc[i] for i in indexs]    
%f positives -->
sum(positives)


indexs count nip --> 279 (<class 'int'>)
positives --> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)


0

In [92]:
# prediction 結果跟 dataset 比對。用 prediction 1 的 index 去算 dataset 的 sum 就是 TP
indexs = list(df_predict_holdout.query('prediction == 1')['row_id'])
%f indexs count nip -->
positives = [data[project_name].iloc[i] for i in indexs]    
%f positives -->
sum(positives)


indexs count nip --> 4 (<class 'int'>)
positives --> [1, 1, 1, 1] (<class 'list'>)


4